In [1]:
from glob import glob
import os
import pandas as pd
from tqdm import tqdm
import math
import re

In [2]:
file_list = glob('./*[A-z].csv')
file_list

['.\\youtube.csv']

In [3]:
file_list = file_list[-1]
file_list

'.\\youtube.csv'

In [3]:
def find_keyword(df) :
    df1 = df.drop_duplicates().dropna(thresh=3,axis=0).reset_index(drop=True)
    for i in range(len(df1)) :
        if pd.isna(df1['user_text'][i]) :
            df1['user_text'][i] = df1['title'][i]
        
    df1['keyword'] = "없음"
    df2 = df1

    for i in range(len(df2)) :      
            try :

                if "재명" in df2['title'][i] :
                    df2['keyword'][i] = "이재명"
#                     print("이재명 :", df2['title'][i], df2['keyword'][i])
            except :
                print(i,  df2['title'][i])
    df3 = df2.reset_index(drop=True)
    df2 = df1

    for j in range(len(df2)) : 

            if "낙연" in df2['title'][j] :
                df2['keyword'][j] = "이낙연"
#                 print("이낙연 :", df2['title'][j], df2['keyword'][j])

    df3 = df3.append(df2).reset_index(drop=True)
    df2 = df1

    for k in range(len(df2)) : 

            if "세균" in df2['title'][k] or "정세균" in df2['title'][k] :
                df2['keyword'][k] = "정세균"
#                 print("정세균 :", df2['title'][k], df2['keyword'][k])

    df3 = df3.append(df2).reset_index(drop=True)
    df3 = df3.drop_duplicates()
    df3.sort_values('date')
    df3.reset_index(drop=True, inplace=True)
#     df3.to_csv(f"{name}2.csv",index=False, encoding='utf-8-sig')
    
    return df3

In [4]:
# STEP 01 키워드 찾기
for file in file_list :
    if 'twitter' in file or 'youtube' in file :
        pass
    else :
        df_new = find_keyword(pd.read_csv(file))
        df_new.to_csv(f"{file[:-4]}_01.csv", index=False, encoding='utf-8-sig')
        print(file,len(df_new) ,  len(df_new[df_new['keyword'] == "이낙연"]), len(df_new[df_new['keyword'] == "이재명"]), len(df_new[df_new['keyword'] == "정세균"]))

In [15]:
com_list = glob('./*_01.csv')
com_list.append('youtube_issue.csv')
com_list

['youtube_issue.csv']

In [16]:
def youtube_comment(df1) :

    # 유튜브

    # comment_keyword = []
    comment_channel = []
    comment_url = []
    comment_date = []
    comment_text = []

    for i in tqdm(range(len(df1))) :
        comments = df1['comment'][i].replace("'","").replace('"','').replace(" ","")[1:-1].split(",")
        for comment in comments :
            comment_channel.append(df1['channel'][i])
            comment_url.append(df1['url'][i])
            comment_date.append(df1['date'][i])
            comment_text.append(comment.split('::')[-1].strip())

    new_df = pd.DataFrame()
    new_df['channel'] = pd.Series(comment_channel)
    new_df['url'] = pd.Series(comment_url)
    new_df['date'] = pd.Series(comment_date) 
    new_df['user_text'] = pd.Series(comment_text)

    # new_df['keyword'] = "4주년연설"

    return new_df

In [17]:
def extract_comments(df1) :


    ## 딴지, 클리앙

    df4 = df1

    keyword = []
    user_name = []
    date = []
    user_text = []

    for i in tqdm(range(len(df4))) :
        comments = df4['com_text'][i][1:-1].split("', '")
    #     print(comments)
        for comment in comments :

            try :            

                target = comment.replace("\n","").split(' :: ')

                if len(target) < 2 :
                    raise Exception("파생구문 확인")
                    break

#                 print("원문 :", target)
#                 print("keyword :", df4['keyword'][i])
#                 print("user_name :", target[0].replace("작성자 : ",""))
#                 print("date :", df4['date'][i])
#                 print("user_text :", target[1])
#                 print("\n")

                keyword.append(df4['keyword'][i])
                user_name.append(target[0].replace("작성자 : ",""))
                date.append(df4['date'][i])
                user_text.append(target[1].replace("\n","").strip())

            except :

                try :
                    target = comment.split(' : ')

                    if len(target) < 2 :
#                         print("기준 미적합 :", target, "\n")
                        break

#                     print("원문 :", target)
#                     print("keyword :", df4['keyword'][i])
#                     print("user_name :", target[0].split(' 2021')[0].replace("작성자 : ",""))
#                     print("date :", df4['date'][i])
#                     print("user_text :", target[1])
#                     print("\n")

                    keyword.append(df4['keyword'][i])
                    user_name.append(target[0].split(' 2021')[0].replace("작성자 : ",""))
                    date.append(df4['date'][i])
                    user_text.append(target[1].replace("\n","").strip())
                except :

                    pass


    new_df = pd.DataFrame()
    new_df['keyword'] = pd.Series(keyword)
    new_df['user_name'] = pd.Series(user_name)
    new_df['date'] = pd.Series(date) 
    new_df['user_text'] = pd.Series(user_text)

    return new_df

In [18]:
for com in com_list :
    name = re.sub(r"[^a-zA-Z0-9]","",com).replace("csv","").replace("01","")
    print(f"현재 {name} 진행 중...")
    if "youtube" in com :
        df_new = youtube_comment(pd.read_csv(com))
    else :
        df_new = extract_comments(pd.read_csv(com))
    
    df_new['domain'] = name
    df_new.to_csv(f"{name}_02.csv", index=False, encoding='utf-8-sig')

100%|█████████████████████████████████████████████████████████████████████████████████| 37/37 [00:00<00:00, 284.96it/s]

현재 youtubeissue 진행 중...


In [9]:
df = pd.read_csv("youtube_02.csv")
df.channel.value_counts()

[공식] 새날      10096
알리미 황희두       3378
김용민TV         2897
딴지방송국         1959
이동형TV         1838
박시영TV          956
김성회의 옳은소리      665
Name: channel, dtype: int64

In [9]:
new_list = glob('./*_01.csv')
# new_list = new_list[:-1]
new_list

['.\\82cook_01.csv',
 '.\\bbombbu_01.csv',
 '.\\clien_01.csv',
 '.\\ddanzi_01.csv',
 '.\\fm_01.csv']

In [11]:
df = pd.DataFrame()
print("(1/2) 커뮤니티 데이터를 합치는 중입니다...")
for new in tqdm(new_list) :
    name = re.sub(r"[^a-zA-Z0-9]","",new).replace("csv","").replace("01","")
    
    for i in ["_01","_02"] :
        df1 = pd.read_csv(f"{name+i}.csv")
        df = df.append(df1).reset_index(drop=True)

print("(2/2) 데이터 클리닝을 진행 중입니다...")

for j in tqdm(range(len(df))) :
    if pd.isna(df['user_text'][j]) :
        try :
            df['user_text'][j] = df['title'][j]
            if len(df['user_text'][j]) < 2 :
                df['user_text'][j] = "image"
#                 df.drop(df.index[j])
                
        except :
            df['user_text'][j] = "image"
#             df.drop(df.index[j])
    try :

        if pd.isna(df['user_name'][j]) :
            df['user_name'][j] = df['writer'][j]
    except :
        pass
    
df.to_csv("community.csv", index=False, encoding='utf-8-sig')
df

  0%|                                                                                            | 0/5 [00:00<?, ?it/s]

(1/2) 커뮤니티 데이터를 합치는 중입니다...


  2%|█▌                                                                     | 12145/571768 [00:00<00:04, 121332.67it/s]

(2/2) 데이터 클리닝을 진행 중입니다...


  9%|██████▏                                                                | 49647/571768 [00:00<00:04, 116414.73it/s]<ipython-input-11-5ee84c545678>:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['user_text'][j] = df['title'][j]
<ipython-input-11-5ee84c545678>:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['user_text'][j] = "image"
100%|███████████████████████████████████████████████████████████████████████| 571768/571768 [00:29<00:00, 19187.85it/s]


,domain,title,user_name,url,date,view_cnt,user_text,com_cnt,com_user,com_text,keyword,like_cnt
0,82cook,남편이 냉장고 문만 열어도 싫으네요. 13,........,https://www.82cook.com/entiz/read.php?bn=15&nu...,2021-05-28,"2,645",진짜 하다하다 냉장고 문여는 것도 꼴보기 싫어지는 날이 오네요.냉장고 문열고 뒤적...,13.0,"['..', '..', '..', '..', '..', '..', '..', '.....","['작성자 : .. :: 참 안타깝네요', '작성자 : .. :: 그럴수도 있어요....",없음,NaN
1,82cook,남편이 열 받게 해서요 16,머리삭발,https://www.82cook.com/entiz/read.php?bn=15&nu...,2021-05-28,"2,351",제목 그대로 남편이 성질 못된데다 열 받게 해서 제 머리를 쥐 파억은듯이 ...,16.0,"['??', '??', '??', '??', '??', '??', '??', '??...","['작성자 : ?? :: 남편이 원글님 머리카락을 잘랐다구요?', '작성자 : ??...",없음,NaN
2,82cook,이승기 팬들이 연애반대 트럭시위 했네요 23,..,https://www.82cook.com/entiz/read.php?bn=15&nu...,2021-05-28,"3,381",팬들이 기획사 상대로 트럭시위하는 건 봤어도 성북동에 열애 반대 트럭시위라니 역대...,23.0,"['..', '..', '..', '..', '..', '..', '..', '.....","['작성자 : .. :: 이승기 안타깝김하죠', '작성자 : .. :: 오버죠미디어...",없음,NaN
3,82cook,종합소득세 몰라서 신고를 못하면 어떻게 되나요 10,ㅇㅇ,https://www.82cook.com/entiz/read.php?bn=15&nu...,2021-05-28,"1,234",프리랜서 사업소득으로 연 천이백을 벌었는데요같은 입장인 사람들이 자기는 홈택스 신...,10.0,"['T ', '글쓴이', 'T ', 'T ', '글쓴이', '글쓴이', 'T ', ...",['작성자 : T :: 원천징수는 잡히니.. 나중에 가산세 내면 되겠네요.그런데 ...,없음,NaN
4,82cook,인경기는 어느 정도 공부해야 가나요 7,...,https://www.82cook.com/entiz/read.php?bn=15&nu...,2021-05-28,"1,077","인서울이면 당연히 좋겠지만요.인경기 아주대, 인가경 정도는 어느 정도 공부해야 가나요?",7.0,"['..', '..', '..', '..', '..', '..', '..']","['작성자 : .. :: 누군가는 쉽고 누군가는 어렵고', '작성자 : .. :: ...",없음,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
571763,fm,NaN,'집에가,NaN,2021-06-02,NaN,지금은 유사 국가입니다만?,NaN,NaN,NaN,정세균,NaN
571764,fm,NaN,준위대학교,NaN,2021-06-02,NaN,집에가 개빡쳐서 안될듯;; 오늘밤에 오랜만에 체육관 갔다와야지',NaN,NaN,NaN,정세균,NaN
571765,fm,NaN,'ARSsumit,NaN,2021-06-02,NaN,동전 뒤집듯이 말 바꾸는 세균,NaN,NaN,NaN,정세균,NaN
571766,fm,NaN,희락생,NaN,2021-06-02,NaN,저 새끼들 말바꾸는 거 보소,NaN,NaN,NaN,정세균,NaN


In [12]:
df.drop_duplicates().dropna(axis=1).reset_index(drop=True)

,domain,date,user_text,keyword
0,82cook,2021-05-28,진짜 하다하다 냉장고 문여는 것도 꼴보기 싫어지는 날이 오네요.냉장고 문열고 뒤적...,없음
1,82cook,2021-05-28,제목 그대로 남편이 성질 못된데다 열 받게 해서 제 머리를 쥐 파억은듯이 ...,없음
2,82cook,2021-05-28,팬들이 기획사 상대로 트럭시위하는 건 봤어도 성북동에 열애 반대 트럭시위라니 역대...,없음
3,82cook,2021-05-28,프리랜서 사업소득으로 연 천이백을 벌었는데요같은 입장인 사람들이 자기는 홈택스 신...,없음
4,82cook,2021-05-28,"인서울이면 당연히 좋겠지만요.인경기 아주대, 인가경 정도는 어느 정도 공부해야 가나요?",없음
...,...,...,...,...
564327,fm,2021-06-02,지금은 유사 국가입니다만?,정세균
564328,fm,2021-06-02,집에가 개빡쳐서 안될듯;; 오늘밤에 오랜만에 체육관 갔다와야지',정세균
564329,fm,2021-06-02,동전 뒤집듯이 말 바꾸는 세균,정세균
564330,fm,2021-06-02,저 새끼들 말바꾸는 거 보소,정세균


In [13]:
df = df.drop_duplicates().dropna(axis=1).reset_index(drop=True)
df.to_csv("community.csv", index=False, encoding='utf-8-sig')

In [56]:
df = pd.read_csv("twitter.csv")
df.keyword.unique()

array(['이낙연', '이재명', '정세균', '이광재', '추미애', '양승조'], dtype=object)

In [54]:
print(len(df[df['keyword']=='이낙연']),len(df[df['keyword']=='이재명']),len(df[df['keyword']=='정세균']))

3262 3944 1065


In [22]:
df = pd.read_csv("youtube.csv")
df

,domain,title,channel,date,url,sub_cnt,view,like,dislike,like_ratio,com_cnt,com_ratio,commentor,comment
0,youtube,[지금 헤어지러 갑니다] ep.43 내겐 너무 예민한 아내,딴지방송국,2021-06-30,https://www.youtube.com/watch?v=JoJMCckT_Ms,828000,35548,2317,18,0.99,196,0.005514,"['드카라바', '파스텔톤', 'Oha -', 'Voyager1 Park', 'Ma...",['최고의 해법을 주셨네요. 사연자분은 저분들 말씀처럼 하시기만 하면 됩니다. 빨리...
1,youtube,"김어준의 다스뵈이다 167회 X파일, 추미애, 대통령 1호기 라면",딴지방송국,2021-06-25,https://www.youtube.com/watch?v=kFGS0VqOq_o,828000,852474,64350,758,0.99,3070,0.003601,"['kim허당철학', 'Seung Jae kim', 'Jang JS', '호야호야호...",['난 추미애 대통령 후보 코어지지자 이다. ~~!!!\n사람이 높은 세상 ~~~ ...
2,youtube,[김어준의 다스뵈이다] 167회 피아니스트 임현정,딴지방송국,2021-06-25,https://www.youtube.com/watch?v=G5JSqmag6ZA,828000,25654,3243,6,1.00,116,0.004522,"['유니허니', '하바나', '정치스타', '김정순', '김전업주부', '평화를 빕...",['귀에 들리는건 분명 피아노 음인데\n머릿속에 그려지는 것은 풍경이고\n드라마네'...
3,youtube,"[짤] 김경률 면접위원, 조응천이 추천",[공식] 새날,2021-07-01,https://www.youtube.com/watch?v=HqcvuVCURQc,395000,2144,297,1,1.00,34,0.015858,"['강승천', '희망의나라', '그로테', '송기만', '정기수', '송기만', '...","['조국 장관 두번 죽이려고 횟집 주인이 장난쳤군', '이것들이 미치지 않고는..."
4,youtube,[본] 김경률 면접관은 조응천 작품?/ 윤석열 지지율 출마 효과 없었다/ 일본을 이...,[공식] 새날,2021-07-01,https://www.youtube.com/watch?v=HueL1PX0Nds,395000,103143,12953,129,0.99,279,0.002705,"['Jang Soo Shin', 'cohahth나', '천미경', 'ai o', '...",['대변인이 이소영 ?? 풉~~\n거기다가 다음 타자가 유인태 ??\n뒤에서 뻘짓거...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
81,youtube,#195 '안녕들하십니까'의 주인공 이대남을 말하다,박시영TV,2021-07-01,https://www.youtube.com/watch?v=a5nMBPe_ROc,197000,14300,2137,7,1.00,7,0.000490,"['순뚱무냥', 'songboy59', 'Gostock Kim', '배부른 소크라테...","['고인물 은근 유용한방송이네요 좋은정보 감사합니다', '민주당의 기득권 보수화 현..."
82,youtube,"#194 추미애, 확실히 깔때기 늘었다! 이탄희 ""윤석열은 사람 살리는 일에 안맞아!""",박시영TV,2021-06-30,https://www.youtube.com/watch?v=gplwHZv5lO8,197000,86864,12784,162,0.99,285,0.003281,"['Rocket R', 'NONE ONE', '진실', '순봉 정', '깜시', '...",['솔직히 정치인 추미애에게는 관심이 없었습니다. \n그런데 법무부장관이라는 독배를...
83,youtube,#193 'KBS 세대 인식 집중 조사' 집중 분석! ｜청년라떼,박시영TV,2021-06-29,https://www.youtube.com/watch?v=KCTNoPlYONQ,197000,24599,2990,23,0.99,64,0.002602,"['AAB', 'AAB', '하얀돼지', 'KS M', '이니사랑', 'Kyung ...","['<조선일보에서 공개한 ""KBS ‘나쁜 이대남’ 그래프, 응답자 없는 구간을 추정..."
84,youtube,"#192 '천기누설' 20대 대선 3대 특징. 이재정, 그가 있어 든든하다!",박시영TV,2021-06-28,https://www.youtube.com/watch?v=YCM3d-yI4NI,197000,73755,8320,91,0.99,128,0.001735,"['Richard Lee', '이재홍', '이수원', '임은진', '뜨아', '김춘...","['많은 민주당 의원들, 국민을 위해 나서고 국민을 대변하기를 꺼려한다면 왜 국회의..."
